In [1]:
import os
from IPython.display import Markdown, HTML, display
from langchain_groq import ChatGroq
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
# Load environment variables
load_dotenv()

True

In [2]:
# Database credentials
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
# Database URL
db_url = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
db = SQLDatabase.from_uri(db_url)

In [3]:
# Initialize SQLDatabase
try:
    database = SQLDatabase.from_uri(db_url)
    print("Database connection successful.")
except Exception as e:
    print(f"Error connecting to database: {e}")
    exit()

Database connection successful.


In [4]:
# Initialize the Groq model
llm = ChatGroq(temperature=0, model_name="llama-3.3-70b-versatile")

In [5]:
# Define the agent instructions and format
MSSQL_AGENT_PREFIX = """
You are an agent designed to interact with a SQL database.
## Instructions:
- Given an input question, create a syntactically correct {dialect} query
to run, then look at the results of the query and return the answer.
- Unless the user specifies a specific number of examples they wish to
obtain, **ALWAYS** limit your query to at most {top_k} results.
- You can order the results by a relevant column to return the most
interesting examples in the database.
- Never query for all the columns from a specific table, only ask for
the relevant columns given the question.
- You have access to tools for interacting with the database.
- You MUST double check your query before executing it. If you get an error
while executing a query, rewrite the query and try again.
- You can also make any DML statements (INSERT, UPDATE, DELETE, DROP etc.)
to the database.
- You can also make any DDL statements (CREATE, ALTER, DROP etc.)
to the database.
- DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS
OF THE CALCULATIONS YOU HAVE DONE.
- Your response should be in Markdown. However, **when running a SQL Query
in "Action Input", do not include the markdown backticks**.
Those are only for formatting the response, not for executing the command.
- ALWAYS, as part of your final answer, explain how you got to the answer
on a section that starts with: "Explanation:". Include the SQL query as
part of the explanation section.
- If the question does not seem related to the database, just return
"I don\'t know" as the answer.
- Only use the below tools. Only use the information returned by the
below tools to construct your query and final answer.
- Do not make up table names, only use the tables returned by any of the
tools below.

## Tools:
"""

MSSQL_AGENT_FORMAT_INSTRUCTIONS = """
## Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do.
Action: the action to take, should be one of [{tool_names}].
Action Input: the input to the action.
Observation: the result of the action.
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer.
Final Answer: the final answer to the original input question.
"""

In [6]:
toolkit=SQLDatabaseToolkit(db=db, llm=llm)

In [7]:
# Create the SQL agent
agent_executor_SQL = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions=MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=llm,
    toolkit=toolkit,
    top_k=30,
    verbose=True
)


In [8]:
# Define the question
QUESTION = """How many tables are in the Database ?"""
# Invoke the agent to answer the question
agent_executor_SQL.invoke(QUESTION)



> Entering new SQL Agent Executor chain...
Thought: To find out how many tables are in the database, I should use the sql_db_list_tables tool to get a list of all tables in the database.
Action: sql_db_list_tables
Action Input: customers, orders, test, titanic## Step 1: Determine the number of tables in the database
The observation from the sql_db_list_tables tool is a comma-separated list of tables in the database. To find the number of tables, we can count the number of items in this list.

## Step 2: Count the number of tables
The list of tables is: customers, orders, test, titanic. We can count the number of items in this list to determine the number of tables in the database.

## Step 3: Determine the final answer
By counting the number of items in the list, we find that there are 4 tables in the database.

Thought: I now know the final answer.
Final Answer: There are 4 tables in the database. 

Explanation: 
To find the number of tables in the database, we used the sql_db_list_

{'input': 'How many tables are in the Database ?',
 'output': 'There are 4 tables in the database. \n\nExplanation: \nTo find the number of tables in the database, we used the sql_db_list_tables tool with no input. The output of this tool was a comma-separated list of tables in the database. We then counted the number of items in this list to determine the number of tables. The SQL query was not necessary in this case, as the sql_db_list_tables tool provided the required information. \n\nNo SQL query was executed in this case, as the sql_db_list_tables tool provided the necessary information.'}